In [3]:
from tweepy import Stream
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import argparse
import string
from twython import Twython  
import json
from sklearn import preprocessing
import gensim 
import re, string
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import re
import os
import sys

In [37]:
def get_latest_tweets(data_dir, auth, time_limit, topic):
    file_name = "stream"
    print("streaming tweets...")
    twitter_stream = Stream(auth, MyListener(data_dir, topic, time_limit),tweet_mode='extended')
    print("tweets obtained and now filtering...")
    twitter_stream.filter(track= topic) # list of querries to track
    data = pd.read_json(data_dir+file_name+".json",lines=True)
    tweets=pd.DataFrame(columns=['time','tweet'],index=data.index)
    no_data = False
    if data.empty:
        no_data = True
    else:
        data = data[data.lang=='en']
        #To get the full-text of the tweet
        for i in data.index:  
               tweets.time[i] = data.created_at[i]
               if pd.isnull(data.retweeted_status[i]):
                     if pd.isnull(data.extended_tweet[i]):
                            tweets.tweet[i] = data.text[i]
                     else:   
                        if "full_text" in data.extended_tweet[i].keys():
                             tweets.tweet[i]=data.extended_tweet[i]["full_text"]

                        else:
                             tweets.tweet[i]=data.text[i] 
               else:
                    if 'extended_tweet' in data.retweeted_status[i].keys():
                        if "full_text" in data.retweeted_status[i]['extended_tweet'].keys():
                            tweets.tweet[i]= data.retweeted_status[i]['extended_tweet']["full_text"]
                    else:
                         tweets.tweet[i] = data.retweeted_status[i]['text']     
        tweets = tweets.sort_values('time', ascending=False)
        tweets=tweets.drop_duplicates()
        tweets.dropna(subset=['tweet'])
    print('tweets are filtered', tweets)
    return tweets, no_data

    

In [5]:
#MyListener() saves the data into a .json file with name stream_query
class MyListener(StreamListener):
    """Custom StreamListener for streaming data."""

    def __init__(self, data_dir, query, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        #query_fname = format_filename(query)
        self.saveFile = open(data_dir+"stream.json", 'a')
        super(MyListener, self).__init__()

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.saveFile.write(data)
            return True
        else:
            self.saveFile.close()
            return False
            

    def on_error(self, status):
        print(status)
        return True


In [6]:
def format_filename(fname):
    """Convert file name into a safe string.
    Arguments:
        fname -- the file name to convert
    Return:
        String -- converted file name
    """
    return ''.join(convert_valid(one_char) for one_char in fname)


def convert_valid(one_char):
    """Convert a character into '_' if invalid.
    Arguments:
        one_char -- the char to convert
    Return:
        Character -- converted char
    """
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

In [126]:
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

 
    
def fully_preprocess(text):
     tic = time.time()
     return  \
     remove_hyphens( \
     remove_urls( \
     remove_mentions( \
     remove_hashtags( \
     remove_twitter_reserved_words( \
     remove_single_letter_words( \
     remove_stopwords( \
     remove_numbers(  \
     strip_html_tags(  \
     remove_accented_chars( \
     expand_contractions(  \
     lemmatize_text(  \
     remove_special_characters(  \
     remove_stopwords(  \
     remove_blank_spaces( \
     Lower(text)))))))))))))))), time.time()-tic
    
def Lower(text):
    tic=time.time()
    text = text.lower()
    toc=time.time()
    print('lower'+str(toc-tic))
    return text
def strip_html_tags(text):
    tic=time.time()
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    toc=time.time()
    print('str_html'+str(toc-tic))
    return text

def remove_accented_chars(text):
    tic=time.time()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    toc=time.time()
    print('remove accented'+str(toc-tic))
    return text

def expand_contractions(text):
    tic=time.time()
    contractions_pattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())), 
                                  flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        tic=time.time()
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = CONTRACTION_MAP.get(match) \
                               if CONTRACTION_MAP.get(match) \
                                else CONTRACTION_MAP.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    text = re.sub("'", "", expanded_text)
    toc=time.time()
    print('expand_contractions'+str(toc-tic))
    return text

# # Removing Special Characters
def remove_special_characters(text):
    tic=time.time()
    text = re.sub('[^a-zA-Z0-9\s]', ' ', text)
    toc=time.time()
    print('remove special'+str(toc-tic))
    return text


# # Lemmatizing text
def lemmatize_text(text):
    tic=time.time()
    nlp = spacy.load('en', parse = False, tag=False, entity=False)
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    toc=time.time()
    print('lemmatize'+str(toc-tic))
    return text
# # Removing Stopwords
#     def remove_stopwords(self):
#         tic=time.time()
#         tokens = tokenizer.tokenize(self.text)
#         tokens = [token.strip() for token in tokens]
#         filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
#         self.text = ' '.join(filtered_tokens) 
#         toc=time.time()
#         print('remove_stop'+str(toc-tic))
#         return self
def remove_stopwords(text, extra_stopwords=None):
    tic=time.time()
    if extra_stopwords is None:
        extra_stopwords = []
    text = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    new_sentence = []
    for w in text:
        if w not in stop_words and w not in extra_stopwords:
            new_sentence.append(w)
    text = ' '.join(new_sentence)
    toc=time.time()
    print('remove_stop'+str(toc-tic))
    return text
def remove_hyphens(text):
    tic=time.time()
    text=text.replace("-"," ")
    toc=time.time()
    print('remove_hyphens'+str(toc-tic))
    return text
def remove_urls(text):
    tic=time.time()
    text = re.sub(pattern=re.compile(
    r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
    r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})'), repl='', string=text)
    toc=time.time()
    print('remove_urls'+str(toc-tic))
    return text

#     def remove_punctuation(self):
#         self.text = self.text.translate(str.maketrans('', '', string.punctuation))
#         return self

def remove_mentions(text):
    tic=time.time()
    text = re.sub(pattern=re.compile(r'@\w*'), repl='', string=text)
    toc=time.time()
    print('remove_mentions'+str(toc-tic))
    return text

def remove_hashtags(text):
    tic=time.time()
    text = re.sub(pattern=re.compile(r'#*'),repl='',string=text)
    toc=time.time()
    print('remove_hashtags'+str(toc-tic))
    return text
def remove_twitter_reserved_words(text):
    tic=time.time()
    text = re.sub(r'\brt\b','',text,flags=re.IGNORECASE) #removing rt
    text = re.sub(r'\bvia\b','',text,flags=re.IGNORECASE) #removing via
    toc=time.time()
    print('remove_twitter_reserved'+str(toc-tic))
    return text

def remove_single_letter_words(text):
    tic=time.time()
    text = re.sub(pattern=re.compile(r'(?<![\w\-])\w(?![\w\-])'),repl=' ',string=text)
    toc=time.time()
    print('remove_single_letter'+str(toc-tic))
    return text
def remove_numbers(text, preserve_years=False):
    tic=time.time()
    text_list = text.split(' ')
    for text in text_list:
        if text.isnumeric():
            if preserve_years:
                if utils.is_year(text):
                    text_list.remove(text)
            else:
                text_list.remove(text)

    text = ' '.join(text_list)
    toc=time.time()
    print('remove_numbers'+str(toc-tic))
    return text
def remove_blank_spaces(text):
    tic=time.time()
    text = re.sub(pattern=re.compile(r'\s{2,}|\t'),repl='',string=text)
    toc=time.time()
    print('remove_blank'+str(toc-tic))
    return text


In [128]:
fully_preprocess(' The same people who were claiming 80% ‘voted')

lower3.0994415283203125e-06
remove_blank2.09808349609375e-05
remove_stop0.0008819103240966797
remove special1.621246337890625e-05
lemmatize0.6699390411376953
expand_contractions0.00011897087097167969
remove accented8.106231689453125e-06
str_html0.00023031234741210938
remove_numbers6.9141387939453125e-06
remove_stop0.0013532638549804688
remove_single_letter1.1205673217773438e-05
remove_twitter_reserved7.867813110351562e-06
remove_hashtags1.0967254638671875e-05
remove_mentions7.152557373046875e-06
remove_urls6.198883056640625e-06
remove_hyphens1.1920928955078125e-06


('people claim vote', 0.708636999130249)

In [130]:
def tokenize_tweet(tweet):
    tokens = [word for word in tweet.lower().split()];
    return(tokens);

def remove_punctuation(tokens):
    clean_words = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens];
    return(clean_words);

def clean_tweet(clean_words):
    stoplist = set(stopwords.words('english'));
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    return(filtered_word_list);


In [133]:
text='49  Labour bigwigs say they now need to commit to a second referendum. Yet in places such as Sunderland it lost 20 per cent of its vote share while the Brexit Party took an extraordinary 40 per cent of the vote. Apparently Labour’s traditional heartlands don’t matter to them anymore!  '
tokens = [tokenize_tweet(word) for word in tweets];
#clean_words
clean_words = remove_punctuation(tokens);
#clean_text
clean_text = clean_text(clean_words);

AttributeError: 'list' object has no attribute 'translate'

In [8]:
def tweet_preprocess(row_tweet):
    return [(TwitterPreprocessor(row_tweet).fully_preprocess().text).split()]

In [88]:
def vectorize_tweet(normalized_tweet):
    vec=np.zeros((300))
    tic=time.time()
    for word in (normalized_tweet and model.vocab):
        vec+=model[word]
    toc=time.time()
    print('time to vectorize one tweet!'+str((toc-tic)/60))
    return preprocessing.normalize(vec.reshape(1,-1))

In [10]:
def vectorize_latest_tweets(tweets, model):
    tweets.tweet= tweets.tweet.astype(str)
    print('preprocessing starting...')
    tic=time.time()
    tweets.loc[:,'normalized'] = tweets.tweet.apply(lambda row_tweet:tweet_preprocess(row_tweet))
    toc=time.time()
    print(str((toc-tic)/60)+'minutes to normalize')
    tic=time.time()
    print('preprocessing finished!')
    tweets.loc[:,'vector'] = tweets.normalized.apply(lambda tweet:vectorize_tweet(tweet))
    toc=time.time()
    print(str((toc-tic)/60)+'minutes to vectorize')
    print('tweets vectorizd!')
    return tweets

In [11]:
def vectorize_user_input(user_input, model):
    
    normalized = tweet_preprocess(user_input)
    word_vec = vectorize_tweet(normalized)
    vectorized_input = {'raw_input': user_input, 'normalized': normalized, 'vector': word_vec}
    return vectorized_input
                    
        

In [35]:
def find_most_similar_tweets(vectorized_input, vectorized_tweets, topn):
    vec_tweets=np.vstack(vectorized_tweets.vector.apply(lambda x: x.tolist()))
    cos=model.cosine_similarities(vectorized_input['vector'].reshape(-1,), vec_tweets)
    vectorized_tweets.loc[:,'similarity_score'] = cos #np.round(cos,10)
    #decimals = pd.Series([8], index=['similarity_score'])
    #vectorized_tweets.round(decimals)
    vectorized_tweets = vectorized_tweets.sort_values(by='similarity_score', ascending=False)
    return vectorized_tweets[0:topn]

In [18]:
def process_user_input(user_input, time_limit, topic):

    track_list=[k for k in topic.split(',')]
    file_name = "stream"
    if os.path.exists(data_dir+file_name+'.json'):
        os.remove(data_dir+file_name+'.json')
    latest_tweets, no_data = get_latest_tweets(data_dir, auth, time_limit, track_list)
    if no_data:
        return'There is no data with topic: '+topic+' in  '+ str(time_limit)+' seconds'
    else:
        latest_tweets = latest_tweets[0:1]
        vectorized_tweets = vectorize_latest_tweets(latest_tweets, model)
        vectorized_user_input = vectorize_user_input(user_input, model)
        #find the top topn= 10  similar tweets
        recommendations = find_most_similar_tweets(vectorized_user_input, vectorized_tweets,topn=10)
    return recommendations
    

In [ ]:
#downloaded pretrained model
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [81]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
     credentials = json.load(file)
# Instantiate an object
python_tweets = Twython( credentials['CONSUMER_KEY'],  credentials['CONSUMER_SECRET'])
auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])

In [93]:
data_dir = '/Users/shahla/Dropbox/SharpestMinds/stream/data/'
user_input = 'Canadian retaliatory tariffs lifted as U.S. kills steel aluminum penalties'
topic = 'brexit' #it can be a list of topics,  comman means 'or'
time_limit=5
user_input = 'Canadian retaliatory tariffs lifted as U.S. kills steel aluminum penalties'
#recommendations = process_user_input(user_input, time_limit, topic)
track_list=[k for k in topic.split(',')]
file_name = "stream"

latest_tweets, no_data = get_latest_tweets(data_dir, auth, time_limit, track_list)

#latest_tweets = latest_tweets[0:1]
# vectorized_tweets = vectorize_latest_tweets(latest_tweets, model)
# vectorized_user_input = vectorize_user_input(user_input, model)
# #find the top topn= 10  similar tweets
# recommendations = find_most_similar_tweets(vectorized_user_input, vectorized_tweets,topn=10)


streaming tweets...
tweets obtained and now filtering...
tweets are filtered                     time                                              tweet
101  2019-05-27 19:15:08  @theSNP Just 505,786 vote for the SNP ,    348...
99   2019-05-27 19:15:08  UK: 99% counted.\n\nRemain parties: 40.4%\nHar...
98   2019-05-27 19:15:08  If you want to make @UKLabour listen, take two...
97   2019-05-27 19:15:08  A brilliant graphic from @SkyNews - clearly sh...
88   2019-05-27 19:15:07  @blunted_james Policy has the option for a vot...
80   2019-05-27 19:15:07  Brexit Derangement Syndrome has been on full d...
81   2019-05-27 19:15:07  So in a nutshell\n\nEU 2014 - UKIP won\nGE 201...
83   2019-05-27 19:15:07  Nigel Farage today insisted he could WIN a Gen...
84   2019-05-27 19:15:07  Whatever you think of his politics, this is a ...
85   2019-05-27 19:15:07  Another thought. Brexit Party was clear about ...
86   2019-05-27 19:15:07  @mireille_pouget @Maxwellsnp @AngusMacNeilSNP ...
87   2019-0

,time,tweet
101,2019-05-27 19:15:08,"@theSNP Just 505,786 vote for the SNP , 348..."
99,2019-05-27 19:15:08,UK: 99% counted.\n\nRemain parties: 40.4%\nHar...
98,2019-05-27 19:15:08,"If you want to make @UKLabour listen, take two..."
97,2019-05-27 19:15:08,A brilliant graphic from @SkyNews - clearly sh...
88,2019-05-27 19:15:07,@blunted_james Policy has the option for a vot...
80,2019-05-27 19:15:07,Brexit Derangement Syndrome has been on full d...
81,2019-05-27 19:15:07,So in a nutshell\n\nEU 2014 - UKIP won\nGE 201...
83,2019-05-27 19:15:07,Nigel Farage today insisted he could WIN a Gen...
84,2019-05-27 19:15:07,"Whatever you think of his politics, this is a ..."
85,2019-05-27 19:15:07,Another thought. Brexit Party was clear about ...


In [102]:
tweets = latest_tweets.copy()
tweets= tweets[0:1]
tweets

,time,tweet
101,2019-05-27 19:15:08,"@theSNP Just 505,786 vote for the SNP , 348..."


In [105]:
tweets.tweet= tweets.tweet.astype(str)
print('preprocessing starting...')
tic=time.time()
tweets.loc[:,'normalized'] = tweets.tweet.apply(lambda row_tweet:tweet_preprocess(row_tweet))
toc=time.time()
print(str((toc-tic)/60)+'minutes to normalize')

preprocessing starting...
remove_hyphens3.0994415283203125e-06
remove_urls5.4836273193359375e-05
remove_mentions3.600120544433594e-05
remove_hashtags0.00011491775512695312
remove_twitter_reserved4.482269287109375e-05
remove_single_letter3.504753112792969e-05
remove_stop0.0015201568603515625
remove_numbers1.3113021850585938e-05
str_html0.0010502338409423828
remove accented2.288818359375e-05
expand_contractions0.0005340576171875
lemmatize1.7759349346160889
remove special5.3882598876953125e-05
remove_stop0.0008461475372314453
remove_blank1.5974044799804688e-05
lower9.5367431640625e-07
0.03087511459986369minutes to normalize


In [94]:


tic=time.time()
print('preprocessing finished!')
tweets.loc[:,'vector'] = tweets.normalized.apply(lambda tweet:vectorize_tweet(tweet))
toc=time.time()
print(str((toc-tic)/60)+'minutes to vectorize')
print('tweets vectorizd!')

False

In [91]:
with pd.option_context('display.max_colwidth', 300):
    print (recommendations)

                   time  \
49  2019-05-27 19:06:08   

                                                                                                                                                                                                                                                                                       tweet  \
49  Labour bigwigs say they now need to commit to a second referendum. Yet in places such as Sunderland it lost 20 per cent of its vote share while the Brexit Party took an extraordinary 40 per cent of the vote. Apparently Labour’s traditional heartlands don’t matter to them anymore!   

                                                                                                                                                                                                                           normalized  \
49  [[labour, bigwig, say, need, commit, second, referendum, yet, place, sunderland, lose, per, cent, vote, share, brexit, party, take,

In [120]:
model.cosine_similarities(model[''],model['Iran'].reshape(1,-1))

array([0.6017679], dtype=float32)

In [83]:
track_list=[k for k in topic.split(',')]
latest_tweets, no_data = get_latest_tweets(data_dir, auth, time_limit, track_list)
latest_tweets = latest_tweets[0:1]


streaming tweets...
tweets obtained and now filtering...
tweets are filtered                    time                                              tweet
80  2019-05-27 15:33:34  Oh dear According to Electoral Calculus, if la...
79  2019-05-27 15:33:34  I wonder why Anas might be saying something li...
78  2019-05-27 15:33:34  EU election results as of right now:\n\nNigel ...
70  2019-05-27 15:33:33                                   Which September?
64  2019-05-27 15:33:33  The Brexit Party Could be Stopped on June 6th ...
65  2019-05-27 15:33:33  4. Brexit: UK result shows the country is spli...
66  2019-05-27 15:33:33  @SKinnock I had to pinch myself but I agree wi...
67  2019-05-27 15:33:33  So it’s another woeful performance for @Nigel_...
69  2019-05-27 15:33:33  So far... in these elections the combined tall...
71  2019-05-27 15:33:33  POOR old Nigel Farage gets awfully tetchy *aga...
72  2019-05-27 15:33:33  Fuck it! No, sorry I mean Bollocks to Brexit! ...
73  2019-05-27 15:33:33